## 10-fold Cross Validation on the Stanik dataset

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 28.0 MB/s 
     |████████████████████████████████| 451 kB 83.8 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 1.9 MB 77.4 MB/s 
     |████████████████████████████████| 7.6 MB 64.5 MB/s 
     |████████████████████████████████| 10.3 MB 75.7 MB/s 
     |████████████████████████████████| 1.3 MB 69.0 MB/s 
     |████████████████████████████████| 5.5 MB 76.8 MB/s 
     |████████████████████████████████| 182 kB 81.1 MB/s 
     |████████████████████████████████| 168 kB 86.8 MB/s 
     |████████████████████████████████| 182 kB 90.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 168 kB 82.9 MB/s 
     |████████████████████████████████| 166 kB 87.3 MB/s 
     |████████████████████████████████| 166 kB 90.9 MB/s 
     |████████████████████████████

In [ ]:
# Import downsampled dataset
df = pd.read_excel('DATASET_downsampled_test5.xlsx')

df = df.drop(columns = ['Unnamed: 0'])

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       2592 non-null   object
 1   kano_labels  2592 non-null   object
 2   labels       2592 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 60.9+ KB


In [ ]:
# Show the distribution of the labels
print(df['labels'].value_counts())

2    648
1    648
3    648
0    648
Name: labels, dtype: int64


In [ ]:
# Rename the columns so that simple transformers can handle them
train_data = df[['review','labels']]
train_data.columns = ['text', 'labels']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2592 non-null   object
 1   labels  2592 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 40.6+ KB


In [ ]:
# Model implementation and k-fold cross validation with Simpletransformers

# Wrapper function to calculate f1-score with additional parameters
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = None)

# Wrapper function to calculate recall with additional parameters
def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = None)

# Wrapper function to calculate precision with additional parameters
def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = None)

# K-fold cross validation with 10 folds
n = 10
kf = KFold(n_splits=n, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc = []
resultsf1 = []
resultsPrecision = []
resultsRecall = []

model_args = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create the ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf.split(train_data):
  train_df = train_data.iloc[train_index]
  val_df = train_data.iloc[val_index]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)
  print(result)
  
  # Add scores to the respective lists
  resultsAcc.append(result['acc'])
  resultsf1.append(result['f1'])
  resultsPrecision.append(result['precision'])
  resultsRecall.append(result['recall'])

  print("results accuracy",resultsAcc)
  print(f"Mean-Precision: {sum(resultsAcc) / len(resultsAcc)}")
  print('results f1: ', resultsf1)
  print('results precision: ', resultsPrecision)
  print('results recall: ', resultsRecall)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

{'mcc': 0.9845227807466223, 'f1': array([1.        , 0.97986577, 1.        , 0.97345133]), 'acc': 0.9884169884169884, 'recall': array([1.        , 0.98648649, 1.        , 0.96491228]), 'precision': array([1.        , 0.97333333, 1.        , 0.98214286]), 'eval_loss': 0.05991404240172194}
results accuracy [0.7038461538461539, 0.8153846153846154, 0.8957528957528957, 0.9498069498069498, 0.9806949806949807, 0.9806949806949807, 0.9806949806949807, 0.9806949806949807, 0.9884169884169884]
Mean-Precision: 0.9195541695541696
results f1:  [array([0.88111888, 0.69387755, 0.70707071, 0.51908397]), array([0.94117647, 0.71666667, 0.86764706, 0.71875   ]), array([0.94308943, 0.875     , 0.94029851, 0.82706767]), array([0.98550725, 0.90756303, 0.96503497, 0.93220339]), array([0.984375  , 0.97959184, 0.98529412, 0.97196262]), array([0.99130435, 0.97841727, 0.99212598, 0.96350365]), array([0.98529412, 0.97810219, 1.        , 0.95798319]), array([1.        , 0.96428571, 0.99270073, 0.96402878]), array([1

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/292 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'mcc': 0.9845798310401837, 'f1': array([0.98484848, 0.99300699, 0.99115044, 0.98461538]), 'acc': 0.9884169884169884, 'recall': array([0.98484848, 1.        , 1.        , 0.96969697]), 'precision': array([0.98484848, 0.98611111, 0.98245614, 1.        ]), 'eval_loss': 0.06050568427687694}
results accuracy [0.7038461538461539, 0.8153846153846154, 0.8957528957528957, 0.9498069498069498, 0.9806949806949807, 0.9806949806949807, 0.9806949806949807, 0.9806949806949807, 0.9884169884169884, 0.9884169884169884]
Mean-Precision: 0.9264404514404514
results f1:  [array([0.88111888, 0.69387755, 0.70707071, 0.51908397]), array([0.94117647, 0.71666667, 0.86764706, 0.71875   ]), array([0.94308943, 0.875     , 0.94029851, 0.82706767]), array([0.98550725, 0.90756303, 0.96503497, 0.93220339]), array([0.984375  , 0.97959184, 0.98529412, 0.97196262]), array([0.99130435, 0.97841727, 0.99212598, 0.96350365]), array([0.98529412, 0.97810219, 1.        , 0.95798319]), array([1.        , 0.96428571, 0.99270073, 0.

In [ ]:
print("results accuracy",resultsAcc)
print(f"Mean-accuracy: {sum(resultsAcc) / len(resultsAcc)}")
print('results f1: ', resultsf1)
print('results precision: ', resultsPrecision)
print('results recall: ', resultsRecall)

results accuracy [0.7038461538461539, 0.8153846153846154, 0.8957528957528957, 0.9498069498069498, 0.9806949806949807, 0.9806949806949807, 0.9806949806949807, 0.9806949806949807, 0.9884169884169884, 0.9884169884169884]
Mean-accuracy: 0.9264404514404514
results f1:  [array([0.88111888, 0.69387755, 0.70707071, 0.51908397]), array([0.94117647, 0.71666667, 0.86764706, 0.71875   ]), array([0.94308943, 0.875     , 0.94029851, 0.82706767]), array([0.98550725, 0.90756303, 0.96503497, 0.93220339]), array([0.984375  , 0.97959184, 0.98529412, 0.97196262]), array([0.99130435, 0.97841727, 0.99212598, 0.96350365]), array([0.98529412, 0.97810219, 1.        , 0.95798319]), array([1.        , 0.96428571, 0.99270073, 0.96402878]), array([1.        , 0.97986577, 1.        , 0.97345133]), array([0.98484848, 0.99300699, 0.99115044, 0.98461538])]
results precision:  [array([0.8630137 , 0.59302326, 0.72916667, 0.64150943]), array([0.92753623, 0.71666667, 0.89393939, 0.70769231]), array([0.93548387, 0.8358209 

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

def PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1):

  np_resultsPrecision = np.array(resultsPrecision)
  np_resultsRecall = np.array(resultsRecall)
  np_resultsf1  = np.array(resultsf1)

  # Calculate the mean precision for each label
  df_resultsPrecision = pd.DataFrame(np_resultsPrecision, columns=list('BDIP'))
  precision_b_mean = df_resultsPrecision['B'].mean()
  precision_d_mean = df_resultsPrecision['D'].mean()
  precision_i_mean = df_resultsPrecision['I'].mean()
  precision_p_mean = df_resultsPrecision['P'].mean()

  # Calculate the mean recall for each label
  df_resultsRecall = pd.DataFrame(np_resultsRecall, columns=list('BDIP'))
  recall_b_mean = df_resultsRecall['B'].mean()
  recall_d_mean = df_resultsRecall['D'].mean()
  recall_i_mean = df_resultsRecall['I'].mean()
  recall_p_mean = df_resultsRecall['P'].mean()

  # Calculate the mean f1-score for each label
  df_resultsf1 = pd.DataFrame(np_resultsf1, columns=list('BDIP'))
  f1_b_mean = df_resultsf1['B'].mean()
  f1_d_mean = df_resultsf1['D'].mean()
  f1_i_mean = df_resultsf1['I'].mean()
  f1_p_mean = df_resultsf1['P'].mean()

  # Create dataframe with summarized results
  metrics = [{'B': precision_b_mean, 'D': precision_d_mean, 'I': precision_i_mean, 'P':precision_p_mean},
          {'B': recall_b_mean, 'D': recall_d_mean, 'I': recall_i_mean, 'P': recall_p_mean},
          {'B': f1_b_mean, 'D': f1_d_mean, 'I': f1_i_mean, 'P': f1_p_mean} ]
  train_df_metrics = pd.DataFrame(metrics, index=['Average precision',
                                'Average recall',
                                'Average F1-Score'])
  return train_df_metrics

PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1)

,B,D,I,P
Average precision,0.963770,0.882208,0.953018,0.906536
Average recall,0.975803,0.936594,0.936011,0.860770
Average F1-Score,0.969671,0.906638,0.944132,0.881265


## Testing on the Brunotte dataset

In [ ]:
# Import of 'Trainingskorpus_Final', which gives the final labeling
testset1 = pd.read_excel('Trainingskorpus_Final.xlsx')

testset1 = testset1.drop(columns = ['Unnamed: 0'])
testset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
dtypes: object(2)
memory usage: 25.5+ KB


In [ ]:
# Show the distribution of the labels
print(testset1['kano_labels'].value_counts())

B    1102
P     395
D      95
I      30
Name: kano_labels, dtype: int64


In [ ]:
# Import of 'Trainingskorpus_InitialLabels', which gives the initial labeling of two sometimes differing labels
testset2 = pd.read_excel('Trainingskorpus_InitialLabels.xlsx')

testset2 = testset2.drop(columns = ['Unnamed: 0'])
testset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   1622 non-null   object
 1   label 1  1622 non-null   object
 2   label 2  1622 non-null   object
dtypes: object(3)
memory usage: 38.1+ KB


In [ ]:
# Merge the two sets so we have the final labels as well as the initial labels in the testset
testset = testset1.merge(testset2, left_on='review', right_on='review')
testset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622 entries, 0 to 1621
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
 2   label 1      1622 non-null   object
 3   label 2      1622 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB


In [ ]:
# Add a column 'equal' which is True if the two initial labels are equal and false otherwise
equal = [None]*np.size(testset['review'])

# Add a column 'equal01' which stores binary values (1 if the two initial labels are equal, 0 otherwise) to calculate the phi-coefficient
equal01 = [None]*np.size(testset['review'])

# Add a column 'labeled_correctly01' which stores binary values (1 if the model correctly labeled the text, 0 otherwise) to calculate the phi-coefficient
labeled_correctly01 = [None]*np.size(testset['review'])

# Add a column 'prediction' which contains the model's predictions
prediction = [None]*np.size(testset['review'])

i = 0
for label1, label2 in zip(testset['label 1'], testset['label 2']):
  if(label1 == label2):
    equal[i] = True
    equal01[i] = 1
  else:
    equal[i] = False
    equal01[i] = 0
  i += 1
  
testset['equal'] = equal

In [ ]:
# Calculate the accuracy of the whole testset as well as 
# only the parts where the inital two labels are equal (accuracy_clear)
# or differ (accuracy_diff)

accuracy_clear = 0.
accuracy_diff = 0.
n_clear = 0
n_diff = 0
all_predicted_labels = []
predicted_labels_equal_initial_label = []
predicted_labels_diff_initial_label = []
true_label_equal = []
true_label_diff = []

# counter to go through all the reviews and fill the column 'labeled_correctly' with true if they have been labeled correctly, and false if not
i = 0

for label, review, equal in zip(testset['kano_labels'],  testset['review'], testset['equal']):
  
  test_prediction, raw_outputs = model.predict([review])
  
  predicted_label = ''

  if test_prediction[0] == 0:
    predicted_label = 'B'
    prediction[i] = 'B'
  if test_prediction[0] == 1:
    predicted_label = 'D'
    prediction[i] = 'D'
  if test_prediction[0] == 2:
    predicted_label = 'I'
    prediction[i] = 'I'
  if test_prediction[0] == 3:
    predicted_label = 'P'
    prediction[i] = 'P'

  all_predicted_labels.append(predicted_label)
  
  if equal:
    n_clear += 1
    predicted_labels_equal_initial_label.append(predicted_label)
    true_label_equal.append(label)
    if label == predicted_label:
      accuracy_clear += 1.

  if not equal:
    n_diff += 1
    predicted_labels_diff_initial_label.append(predicted_label)
    true_label_diff.append(label)
    if label == predicted_label:
      accuracy_diff += 1.

  if label == predicted_label:
    labeled_correctly01[i] = 1
  else:
    labeled_correctly01[i] = 0
  i+=1

testset['labeled_correctly'] = labeled_correctly01
testset['prediction'] = prediction

accuracy = (accuracy_clear + accuracy_diff) / (n_clear + n_diff)

accuracy_clear /= n_clear
accuracy_diff /= n_diff

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
testset.to_excel("Brunotte_with_predictions_test5_bert.xlsx")  

In [ ]:
print('Accuracy Trainingskorpus:                                                  ', round(accuracy*100, 2), '%,  n =', (n_clear + n_diff))
print('Accuracy Trainingskorpus only on the data that coincides on initial labels:', round(accuracy_clear*100, 2), '%,  n =', n_clear)
print('Accuracy Trainingskorpus only on the data that differs on initial labels:  ', round(accuracy_diff*100, 2), '%,  n = ', n_diff)

Accuracy Trainingskorpus:                                                   64.67 %,  n = 1622
Accuracy Trainingskorpus only on the data that coincides on initial labels: 68.26 %,  n = 1383
Accuracy Trainingskorpus only on the data that differs on initial labels:   43.93 %,  n =  239


In [ ]:
# Calculate precision, recall and f1-score for each label

test_df_metrics = precision_recall_fscore_support(testset['kano_labels'], all_predicted_labels, average=None,labels=['B', 'D', 'I', 'P'])

print('Precision B: ', test_df_metrics[0][0])
print('Precision D: ', test_df_metrics[0][1])
print('Precision I: ', test_df_metrics[0][2])
print('Precision P: ', test_df_metrics[0][3])

print('Recall B: ', test_df_metrics[1][0])
print('Recall D: ', test_df_metrics[1][1])
print('Recall I: ', test_df_metrics[1][2])
print('Recall P: ', test_df_metrics[1][3])

print('F1-score B: ', test_df_metrics[2][0])
print('F1-score D: ', test_df_metrics[2][1])
print('F1-score I: ', test_df_metrics[2][2])
print('F1-score P: ', test_df_metrics[2][3])

Precision B:  0.9028290282902829
Precision D:  0.35
Precision I:  0.5714285714285714
Precision P:  0.4020618556701031
Recall B:  0.6660617059891107
Recall D:  0.8105263157894737
Recall I:  0.13333333333333333
Recall P:  0.5924050632911393
F1-score B:  0.766579634464752
F1-score D:  0.4888888888888889
F1-score I:  0.21621621621621623
F1-score P:  0.47901740020470834


In [ ]:
# Calculates the phi-coefficient of the labels being predicted correctly and the researchers being 

print('MCC for initial agreement of raters and correct or false labeling',matthews_corrcoef(labeled_correctly01, equal01))

MCC for initial agreement of raters and correct or false labeling 0.18037915777015082


## 10-fold Cross Validation on a combination of the Stanik dataset and the Brunotte dataset

In [ ]:
# Import combined downsampled dataset 
df_combined = pd.read_excel('DATASET_Trainingskorpus_combined_test5.xlsx')

df_combined = df_combined.drop(columns = ['Unnamed: 0'])

# Rename column so that simple transformers can handle it
df_combined = df_combined.rename(columns = {'review': 'text'})
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2972 non-null   object
 1   kano_labels  2972 non-null   object
 2   labels       2972 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.8+ KB


In [ ]:
# Show the distribution of the labels
print(df_combined['labels'].value_counts())

0    743
1    743
3    743
2    743
Name: labels, dtype: int64


In [ ]:
# 10-fold cross validation on the combined dataset

n2 = 10
kf2 = KFold(n_splits=n2, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc2 = []
resultsf12 = []
resultsPrecision2 = []
resultsRecall2 = []

model_args2 = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create a ClassificationModel
model2 = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf2.split(df_combined):
  train_df2 = df_combined.iloc[train_index]
  val_df2 = df_combined.iloc[val_index]

  model2.train_model(df_combined)
  result2, model_outputs2, wrong_predictions2 = model2.eval_model(val_df2, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)

  print(result2)
  # Add scores to the respective lists
  resultsAcc2.append(result2['acc'])
  resultsf12.append(result2['f1'])
  resultsPrecision2.append(result2['precision'])
  resultsRecall2.append(result2['recall'])

  print("results accuracy",resultsAcc2)
  print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
  print('results f1: ', resultsf12)
  print('results precision: ', resultsPrecision2)
  print('results recall: ', resultsRecall2)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.746030310288087, 'f1': array([0.8976378 , 0.75641026, 0.89473684, 0.70807453]), 'acc': 0.8087248322147651, 'recall': array([0.93442623, 0.80821918, 0.87179487, 0.6627907 ]), 'precision': array([0.86363636, 0.71084337, 0.91891892, 0.76      ]), 'eval_loss': 0.5098026677181846}
results accuracy [0.8087248322147651]
Mean-accuracy: 0.8087248322147651
results f1:  [array([0.8976378 , 0.75641026, 0.89473684, 0.70807453])]
results precision:  [array([0.86363636, 0.71084337, 0.91891892, 0.76      ])]
results recall:  [array([0.93442623, 0.80821918, 0.87179487, 0.6627907 ])]


...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.9955105630960102, 'f1': array([0.99378882, 1.        , 0.992     , 1.        ]), 'acc': 0.9966329966329966, 'recall': array([0.98765432, 1.        , 1.        , 1.        ]), 'precision': array([1.        , 1.        , 0.98412698, 1.        ]), 'eval_loss': 0.03074288054516441}
results accuracy [0.8087248322147651, 0.8859060402684564, 0.9292929292929293, 0.9764309764309764, 0.9932659932659933, 0.9865319865319865, 0.98989898989899, 1.0, 0.9966329966329966, 0.9966329966329966]
Mean-accuracy: 0.956331774117009
results f1:  [array([0.8976378 , 0.75641026, 0.89473684, 0.70807453]), array([0.94936709, 0.86092715, 0.91780822, 0.80851064]), array([0.96551724, 0.88709677, 0.96644295, 0.88435374]), array([0.98412698, 0.96644295, 0.98765432, 0.96815287]), array([0.9924812 , 0.99354839, 1.        , 0.98684211]), array([0.98550725, 0.98571429, 1.        , 0.975     ]), array([0.99393939, 0.97744361, 0.98630137, 1.        ]), array([1., 1., 1., 1.]), array([1.        , 0.99481865, 1.      

In [ ]:
print("results accuracy",resultsAcc2)
print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
print('results f1: ', resultsf12)
print('results precision: ', resultsPrecision2)
print('results recall: ', resultsRecall2)

results accuracy [0.8087248322147651, 0.8859060402684564, 0.9292929292929293, 0.9764309764309764, 0.9932659932659933, 0.9865319865319865, 0.98989898989899, 1.0, 0.9966329966329966, 0.9966329966329966]
Mean-accuracy: 0.9264404514404514
results f1:  [array([0.8976378 , 0.75641026, 0.89473684, 0.70807453]), array([0.94936709, 0.86092715, 0.91780822, 0.80851064]), array([0.96551724, 0.88709677, 0.96644295, 0.88435374]), array([0.98412698, 0.96644295, 0.98765432, 0.96815287]), array([0.9924812 , 0.99354839, 1.        , 0.98684211]), array([0.98550725, 0.98571429, 1.        , 0.975     ]), array([0.99393939, 0.97744361, 0.98630137, 1.        ]), array([1., 1., 1., 1.]), array([1.        , 0.99481865, 1.        , 0.99159664]), array([0.99378882, 1.        , 0.992     , 1.        ])]
results precision:  [array([0.86363636, 0.71084337, 0.91891892, 0.76      ]), array([0.9375    , 0.82278481, 0.97101449, 0.81428571]), array([0.94382022, 0.91666667, 0.96      , 0.89041096]), array([0.96875   , 0.

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

PrecisionRecallF1Table(resultsPrecision2, resultsRecall2, resultsf12)

,B,D,I,P
Average precision,0.968429,0.935773,0.979469,0.942671
Average recall,0.984531,0.949921,0.970255,0.922635
Average F1-Score,0.976237,0.942240,0.974494,0.932253
